# ABS Inflation multi-measure

## Python set-up

In [1]:
# system imports
from pathlib import Path
from typing import Sequence, cast

# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    AbsLandingPage,
    AbsMultiSeries,
    AbsSelectInput,
    AbsSelectionDict,
    df_from_ams,
    get_multi_series,
    rename_cols_with_stype,
)
from plotting import clear_chart_dir, finalise_plot, line_plot, set_chart_dir
from utility import annualise_percentages, percent_change, qtly_to_monthly

In [2]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# save charts in this notebook
plt.style.use("fivethirtyeight")
CHART_DIR = "./CHARTS/Inflation/"
set_chart_dir(CHART_DIR)
clear_chart_dir(CHART_DIR)

# some plotting constants
TARGET = {
    "ymin": 2,
    "ymax": 3,
    "color": "#dddddd",
    "label": "2-3% annual inflation target",
    "zorder": -1,
}
LFOOTER = "Australia. Orig = Original series. SA = Seasonally adjusted series. "

# display charts in this notebook
SHOW = False

## Main comparative charts

### Identify inflation measures across ABS collections

In [3]:
wanted: AbsSelectionDict = {
    # specify the data items we wish to extract from the ABS ...
    "Qrtly All Groups CPI": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Quarter of Previous Year ;",
        search2="All groups CPI, seasonally adjusted ;  Australia",
        abbr="Q-CPI",
        calc_growth=False,
    ),
    "Monthly CPI Indicator": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="All groups CPI, seasonally adjusted ;  Australia",
        abbr="M-CPI",
        calc_growth=False,
    ),
    "Qrtly CPI Trimmed Mean": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Quarter of Previous Year",
        search2="Trimmed Mean ;  Australia",
        abbr="Q-CPI-TM",
        calc_growth=False,
    ),
    "Monthly CPI Trimmed Mean": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="Annual Trimmed Mean ;  Australia ;",
        abbr="M-CPI-TM",
        calc_growth=False,
    ),
    "Producer Price Index": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="producer-price-indexes-australia",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage change from corresponding quarter of previous year",
        search2="Final ;  Total ",
        abbr="PPI",
        calc_growth=False,
    ),
    "Wage Price Index": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="wage-price-index-australia",
        ),
        table="1",
        orig_sa="SA",
        search1="Percentage Change From Corresponding Quarter of Previous Year",
        search2="Australia ;  Total hourly rates of pay excluding bonuses ;  "
        + "Private and Public ;  All industries ;",
        abbr="WPI",
        calc_growth=False,
    ),
    "Households final consumption Price Deflator": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="national-accounts",
            topic="australian-national-accounts-national-income-expenditure-and-product",
        ),
        table="5",
        orig_sa="SA",
        search1=" ",
        search2="Households ;  Final consumption expenditure ;",
        abbr="HFCE",
        calc_growth=True,
    ),
   #"Non-farm hourly employee compensation": AbsSelectInput(
   #     landing_page=AbsLandingPage(
   #         theme="economy",
   #         parent_topic="national-accounts",
   #         topic="australian-national-accounts-national-income-expenditure-and-product",
   #     ),
   #     table="24",
   #     orig_sa="SA",
   #     search1="Current prices ;",
   #     search2="Non-farm compensation of employees per hour:",
   #     abbr="NFHEC",
   #     calc_growth=True,
   # ),
}

In [4]:
dataset = get_multi_series(wanted, verbose=False)

### Multi-measure plot

In [5]:
def plot_multi_inflation(abs_data: AbsMultiSeries):
    """Produce the mulit-mesure inflation charts."""

    frame = df_from_ams(abs_data)
    source = "ABS " + ", ".join(
        [f"{x.cat_id.split(r'.', 1)[0]}-{x.table}" for x in abs_data.values()]
    )
    latest = ", ".join(
        [
            f"{abs_data[name].abbr} {frame[name].dropna().round(1).iloc[-1]}"
            for name in frame.columns
        ]
    )
    frame = rename_cols_with_stype(frame, abs_data)

    starts = ("1959-01-01", "2019-12-01")
    styles = (None, ["solid", "dotted", "dashed"] * 3)
    markers = (
        None,
        ["o", "v", "^", "<", ">", "8", "s", "p", "*", "h", "H", "D", "d", "P", "X"],
    )

    for start, style, marker in zip(starts, styles, markers):
        line_plot(
            frame,
            starts=start,
            style=style,
            marker=marker,
            markersize=6,
            dropna=True,
            title="Inflation measures",
            ylabel="Per Cent Annual Growth",
            legend={"fontsize": "xx-small", 'ncol': 1, "loc": "best",},
            axhspan=TARGET,
            y0=True,
            rfooter=source,
            lfooter=LFOOTER,
            rheader=latest if start != starts[0] else None,
            tags=start if start is not None else "",
            show=SHOW,
        )

In [6]:
plot_multi_inflation(dataset)

### Weighted CPI trajectory

In [7]:
def trajectory(abs_data: AbsMultiSeries) -> None:
    """Produce a CPI trajectory chart."""

    frame = df_from_ams(abs_data)
    frame = rename_cols_with_stype(frame, abs_data)
    thresh = 2.5
    start_year = 2020
    series_name = "Qrtly CPI Trimmed Mean (SA)"
    cpiwm_mon = frame.loc[
        cast(pd.PeriodIndex, frame.index).year >= start_year, series_name
    ].dropna()
    cpiwm_q = cpiwm_mon[cast(pd.PeriodIndex, cpiwm_mon.index).month.isin([3, 6, 9, 12])]
    cpiwm_q.index = (
        cast(pd.PeriodIndex, cpiwm_q.index).to_timestamp(how="end").to_period(freq="Q")
    )
    start = cpiwm_q[cpiwm_q > 2.5].index[0]

    peak = cpiwm_q[cpiwm_q == cpiwm_q.max()].index[0]
    peak_minus, peak_plus = (peak - 1, peak + 1)
    late_inflection = peak + 4
    tail_end = cpiwm_q.index[-1]
    down_from = late_inflection  # or peak_plus

    up_rate = float((cpiwm_q[peak_minus] - cpiwm_q[start]) / (peak_minus - start).n)
    down_rate = float((cpiwm_q[tail_end] - cpiwm_q[down_from]) / (tail_end - down_from).n)
    print(f"{up_rate=}, {down_rate=}")

    count, maximum = -1, 10
    s = pd.Series()
    while True:
        count = count + 1
        pos = down_from + count
        if pos <= tail_end:
            continue
        s[pos] = cpiwm_q[down_from] + (down_rate * count)
        if s[pos] <= thresh or count == maximum:
            break
    s = qtly_to_monthly(s, interpolate=False).dropna()
    ax = cpiwm_mon.plot(label=series_name)
    s.plot(ax=ax, label="Current trajectory")

    for m in [
        "Monthly CPI Trimmed Mean (Orig)",
        # "Monthly CPI Indicator (SA)",
    ]:
        cpi_mon = frame.loc[cast(pd.PeriodIndex, frame.index).year >= start_year, m]
        cpi_mon.plot(ax=ax, lw=2, label=m)

    for x in s.index:
        ax.text(x, s[x], round(s[x], 1))
    ax.text(cpiwm_mon.index[-1], cpiwm_mon.iloc[-1], round(cpiwm_mon.iloc[-1], 1))

    finalise_plot(
        axes=ax,
        title="Current underlying inflation trajectory",
        ylabel="Per cent",
        legend={
            "fontsize": "xx-small",
            "loc": "upper left",
        },
        axhspan=TARGET,
        lfooter=LFOOTER,
        rfooter="Source: ABS 6401, 6484",
        show=SHOW,
    )


trajectory(dataset)

up_rate=1.0999999999999999, down_rate=-0.20000000000000018


### Monthly v Quarterly pairs

In [8]:
def plot_mq(
    ams_data: AbsMultiSeries,
    mq_pairs: dict[str, tuple[str, str]],
) -> None:
    """Plot some monthly v quarterly series."""

    frame = df_from_ams(ams_data)
    frame = rename_cols_with_stype(frame, ams_data)

    start_year = 2019
    for name, (m, q) in mq_pairs.items():
        data_m = frame.loc[cast(pd.PeriodIndex, frame.index).year >= start_year, m]
        data_q = frame.loc[cast(pd.PeriodIndex, frame.index).year >= start_year, q]
        data_q = data_q.interpolate(limit_area="inside", limit=2)
        ax = data_q.plot(label=q)
        data_m.plot(ax=ax, lw=1.5, label=m)

        finalise_plot(
            axes=ax,
            title=f"{name}: Monthly Vs Quarterly",
            ylabel="Per cent",
            legend={
                "fontsize": "xx-small",
                "loc": "upper left",
            },
            axhspan=TARGET,
            lfooter=LFOOTER,
            rfooter="Source: ABS 6401, 6484",
            y0=True,
            show=SHOW,
        )

In [9]:
data_pairs = {
    # "pair name": ["a", "b"],
    "CPI Seasonally Adjusted": (
        "Monthly CPI Indicator (SA)",
        "Qrtly All Groups CPI (SA)",
    ),
    "Trimmed Mean": ("Monthly CPI Trimmed Mean (Orig)", "Qrtly CPI Trimmed Mean (SA)"),
}

plot_mq(dataset, data_pairs)

### Monthly v Quarterly - analytic series

In [10]:
# Datasets
wanted_mq: AbsSelectionDict = {
    # specify the data items we wish to extract from the ABS ...
    "Qrtly All Groups CPI": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Quarter of Previous Year ;",
        search2="All groups CPI ;  Australia",
        abbr="Q-CPI-Orig",
        calc_growth=False,
    ),
    "Monthly CPI Indicator": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="All groups CPI ;  Australia ;",
        abbr="M-CPI-Orig",
        calc_growth=False,
    ),

    "Qrtly CPI services component": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Quarter of Previous Year ;",
        search2="All groups, services component ;  Australia ;",
        abbr="Q-CPI-Orig",
        calc_growth=False,
    ),
    "Monthly CPI services component": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="All groups, services component ;  Australia ;",
        abbr="M-CPI-Orig",
        calc_growth=False,
    ),

    "Qrtly CPI goods component": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Quarter of Previous Year ;",
        search2="All groups, goods component ;  Australia ;",
        abbr="Q-CPI-Orig",
        calc_growth=False,
    ),
    "Monthly CPI goods component": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="All groups, goods component ;  Australia ;",
        abbr="M-CPI-Orig",
        calc_growth=False,
    ),

    "Qrtly CPI tradables": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Quarter of Previous Year ;",
        search2="Tradables ;  Australia ;",
        abbr="Q-CPI-Orig",
        calc_growth=False,
    ),
    "Monthly CPI tradables": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="Tradables ;  Australia ;",
        abbr="M-CPI-Orig",
        calc_growth=False,
    ),

    "Qrtly CPI non-tradables": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Quarter of Previous Year ;",
        search2="Non-tradables ;  Australia ;",
        abbr="Q-CPI-Orig",
        calc_growth=False,
    ),
    "Monthly CPI non-tradables": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="Non-tradables ;  Australia ;",
        abbr="M-CPI-Orig",
        calc_growth=False,
    ),
}


In [11]:


data_pairs_mq = {
    # "pair name": ["a", "b"],
    "CPI Original Series": (
        "Monthly CPI Indicator (Orig)",
        "Qrtly All Groups CPI (Orig)",
    ),
    "CPI Services Component": (
        "Monthly CPI services component (Orig)",
        "Qrtly CPI services component (Orig)",
    ),
    "CPI Goods Component": (
        "Monthly CPI goods component (Orig)",
        "Qrtly CPI goods component (Orig)",
    ),
    "CPI Tradables": (
        "Monthly CPI tradables (Orig)",
        "Qrtly CPI tradables (Orig)",
    ),
    "CPI Non-tradables": (
        "Monthly CPI non-tradables (Orig)",
        "Qrtly CPI non-tradables (Orig)",
    ),
}

In [12]:

dataset_mq = get_multi_series(wanted_mq, verbose=False)
plot_mq(dataset_mq, data_pairs_mq)

## Monthly seasonally adjusted v original

In [13]:
wanted2: AbsSelectionDict = {
    # specify the data items we wish to extract from the ABS ...
    "Qrtly All Groups CPI - SA": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Quarter of Previous Year ;",
        search2="All groups CPI, seasonally adjusted ;  Australia",
        abbr="Q-CPI",
        calc_growth=False,
    ),
    "Monthly CPI Indicator - SA": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="All groups CPI, seasonally adjusted ;  Australia",
        abbr="M-CPI-SA",
        calc_growth=False,
    ),
    "Monthly CPI Indicator - Orig": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="All groups CPI ;  Australia ;",
        abbr="M-CPI-Orig",
        calc_growth=False,
    ),
}

In [14]:
def two_monthly():
    """Produce monthly SA v Orig."""

    dataset2 = get_multi_series(wanted2, verbose=False)
    frame = df_from_ams(dataset2)
    line_plot(
        frame,
        title="Monthly CPI indicators vs Quarterly CPI",
        starts=("2018-12-01",),
        dropna=True,
        ylabel="Through the year growth %",
        width=[3.0, 2.0, 1.0],
        color=[
            "darkorange",
            "cornflowerblue",
            "darkred",
        ],
        axhspan=TARGET,
        lfooter=LFOOTER,
        rfooter="Source: ABS 6401, 6484",
        show=SHOW,
    )

In [15]:
two_monthly()

## Annualised growth

In [16]:
def multi_annualised(
    inputs: AbsSelectionDict,
    title: str,
    periods: Sequence[int] = (3, 3, 6, 6),  # months - period to annualise
    columns: Sequence[int] = (0, 1, 0, 1),  # column numbers - based on inputs order
):
    "Plot annualised growth."
    
    multi_data = get_multi_series(inputs, verbose=False)
    frame3 = (
        # get every month into focus
        df_from_ams(multi_data)
        .copy()
        .to_timestamp()
        .resample("ME")
        .first()  # should only be zero or one
        .to_period(freq="M")
    )
    renamer = {c: f"{c} - ({inputs[c].orig_sa})" for c in frame3.columns}
    frame3 = frame3.rename(columns=renamer)

    growth_frame = pd.DataFrame(index=frame3.index)
    for period, col in zip(periods, columns):
        name = frame3.columns[col]
        index = frame3[name]
        factor = 12.0 / period
        growth = annualise_percentages(percent_change(index, period), factor)
        growth_frame[f"{name} - {period} months annualised"] = growth

    ep_text = ""
    if len(growth_frame.columns) == 2:
        endpoints = {}
        for c in growth_frame.columns:
            endpoints[c.split(" ")[0]] = round(growth_frame[c].dropna().iloc[-1], 2)
        ep_text = " Endpoints: "
        for c, val in endpoints.items():
            ep_text += f"{c}={val} "

    line_plot(
        growth_frame,
        title=title,
        starts=("2018-12-01",),
        dropna=True,
        ylabel="Annualised growth %",
        width=[3.0, 2.0, 1.0, 1.0],
        color=["darkorange", "cornflowerblue", "darkred", "green"],
        axhspan=TARGET,
        lfooter=f"{LFOOTER}{ep_text}",
        rfooter="Source: ABS 6401, 6484",
        show=SHOW,
    )

In [17]:
wanted3sa: AbsSelectionDict = {
    # specify the data items we wish to extract from the ABS ...
    "Qrtly All Groups CPI": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="SA",
        search1=" ",
        search2="Index Numbers ;  All groups CPI, seasonally adjusted ;  Australia ;",
        abbr="-",
        calc_growth=False,
    ),
    "Monthly CPI Indicator": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="SA",
        search1="Index Numbers ;  All groups CPI, seasonally adjusted ;  Australia ;",
        search2=" ",
        abbr="-",
        calc_growth=False,
    ),
}

wanted3orig: AbsSelectionDict = {
    # specify the data items we wish to extract from the ABS ...
    "Qrtly All Groups CPI": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="Orig",
        search1=" ",
        search2="Index Numbers ;  All groups CPI ;  Australia ;",
        abbr="-",
        calc_growth=False,
    ),
    "Monthly CPI Indicator": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Index Numbers ;  All groups CPI ;  Australia ;",
        search2=" ",
        abbr="-",
        calc_growth=False,
    ),
}

In [18]:
multi_annualised(wanted3sa, title="Annualised CPI growth (SA)")
multi_annualised(wanted3orig, title="Annualised CPI growth (Orig)")

In [19]:
# multi-period headline annualised
for n in range(3, 10, 3):
    multi_annualised(
        wanted3sa,
        title=f"{n} Month Annualised Headline CPI growth (SA)",
        periods=(n, n),
        columns=(0, 1),
    )
    multi_annualised(
        wanted3orig,
        title=f"{n} Month Annualised Headline CPI growth (Orig)",
        periods=(n, n),
        columns=(0, 1),
    )

In [20]:
wanted4 = {
    "Qrtly CPI Trimmed Mean": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="SA",
        search1="Index Numbers ;",
        search2="Trimmed Mean ;  Australia",
        abbr="-",
        calc_growth=False,
    ),
    # ABS does not include the monthly Trimmed Mean index ???
    "Monthly CPI cxcluding volatile items and holidays": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="SA",
        search1="Index Numbers ;",
        search2="All groups CPI excluding 'volatile items' and holiday travel ;  Australia ;",
        abbr="-",
        calc_growth=False,
    ),
}

In [21]:
multi_annualised(wanted4, title="Annualised Core CPI growth")

## Finished

In [22]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Apr 24 2024 15:00:53

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.22.2

matplotlib: 3.8.4
pandas    : 2.2.2

Watermark: 2.4.3



In [23]:
print("Finished")

Finished
